# 1.) Preprocess your data into scaled input variables and an output variable


In [1]:
import pandas as pd
#from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [2]:
#drive.mount('/content/gdrive/', force_remount = True)

In [3]:
df = pd.read_csv("/Users/Cade/Downloads/CLV.csv")


In [4]:
df

,Unnamed: 0,Customer Lifetime Value,Income,Number of Policies,Total Claim Amount,Months Since Last Claim,Vehicle Size_Large,Vehicle Size_Medsize,Gender_M,EmploymentStatus_Employed,EmploymentStatus_Medical Leave,EmploymentStatus_Retired,EmploymentStatus_Unemployed,Policy Type_Corporate Auto,Policy Type_Personal Auto,Marital Status_Divorced,Marital Status_Married,Marital Status_Single
0,0,2763.519279,56274,1,384.811147,32,0,1,0,1,0,0,0,1,0,0,1,0
1,1,6979.535903,0,8,1131.464935,13,0,1,0,0,0,0,1,0,1,0,0,1
2,2,12887.431650,48767,2,566.472247,18,0,1,0,1,0,0,0,0,1,0,1,0
3,3,7645.861827,0,7,529.881344,18,0,1,1,0,0,0,1,1,0,0,1,0
4,4,2813.692575,43836,1,138.130879,12,0,1,1,1,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,9129,23405.987980,71941,2,198.234764,18,0,1,1,1,0,0,0,0,1,0,1,0
9130,9130,3096.511217,21604,1,379.200000,14,0,1,0,1,0,0,0,1,0,1,0,0
9131,9131,8163.890428,0,2,790.784983,9,0,1,1,0,0,0,1,1,0,0,0,1
9132,9132,7524.442436,21941,3,691.200000,34,1,0,1,1,0,0,0,0,1,0,1,0


In [5]:
X = df.drop(["Unnamed: 0", "Customer Lifetime Value"], axis =1 )
y = df['Customer Lifetime Value']

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3)

In [7]:
from sklearn.preprocessing import StandardScaler

# 2.) Run a GridSearch CV on at least 10 possible combinations of hyper parameters 

In [28]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

In [31]:
clf = MLPRegressor()

params = {
    "hidden_layer_sizes": [(1,), (2,),(3,),(4,),(5,)], 
    "activation" : ['relu','logistic']
}

In [32]:
grid = GridSearchCV(clf, params, cv = 5)
grid.fit(X_train, y_train)

/Users/Cade/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/Cade/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/Cade/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/Cade/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/Cade/opt/anaconda3/li

GridSearchCV(cv=5, estimator=MLPRegressor(),
             param_grid={'activation': ['relu', 'logistic'],
                         'hidden_layer_sizes': [(1,), (2,), (3,), (4,), (5,)]})

In [33]:
print(grid.best_params_)

{'activation': 'relu', 'hidden_layer_sizes': (5,)}


# 3.) Train a model with the optimal solution from GridSearch

In [34]:
MLPRegressor(**grid.best_params_)

MLPRegressor(hidden_layer_sizes=(5,))

In [46]:
mlp = MLPRegressor(hidden_layer_sizes = (5,), 
             activation = "relu").fit(X_train,y_train)

# 4.) What are the in-sample and out of sample MSEs

In [47]:
y_predict = mlp.predict(X_train)

In [81]:
from sklearn.metrics import mean_squared_error

In [82]:
mse_in = mean_squared_error(y_train, y_predict)

In [83]:
oos_predict = mlp.predict(X_test)
oos_mse = mean_squared_error(y_test, oos_predict)

In [88]:
print("In sample:", mse_in, "Out of sample:", oos_mse)

In sample: 47232571.198168255 Out of sample: 43044582.90058756


# 5.) Build a Keras with the architecture defined by GridSearchCV

In [8]:
import keras.models
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense

2023-02-24 12:40:58.966607: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model = Sequential()
model.add(Dense(5, activation = "relu"))

2023-02-24 12:41:05.878719: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
model.compile(loss = "mse", optimizer = Adam(lr=0.01))
model.fit(X_train, y_train, batch_size=32, epochs = 1)

/Users/Cade/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


200/200 [==============================] - 1s 1ms/step - loss: 102797024.0000


# 6.) Make two visualizations of your NN using “plot_model” and “ann_viz”

## Like we talked in office hours-- graphviz causing issues on both notebook and colab so just ommitting it the plots for now. Thanks for looking it over for me

In [11]:
from tensorflow.keras.utils import plot_model

In [12]:
plot_model(model, show_shapes = True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [13]:
from ann_visualizer.visualize import ann_viz

In [14]:
ann_viz(model, view = True, title = "CLV Keras model")

ModuleNotFoundError: No module named 'graphviz'